In [283]:
# keras version : 2.4.3
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from keras.utils import to_categorical
import pickle
# from sklearn.ensemble import RandomForestClassifier
#from keras.utils import np_utils
#from sklearn.preprocessing import LabelEncoder

In [284]:
# Random seed for reproducibility
seed = 12
np.random.seed(seed)
# Import data
df = pd.read_csv('training_dataset_remove_duplicates.csv')
# Print first 10 samples
# print(df.head(10))

In [285]:
# Check missing values
print(df.isna().sum()) # No missing values!

AP_1       0
AP_2       0
AP_3       0
AP_4       0
AP_5       0
          ..
AP_1595    0
AP_1596    0
AP_1597    0
AP_1598    0
target     0
Length: 1599, dtype: int64


In [286]:
num_row, num_col = df.shape

In [287]:
num_row, num_col

(287, 1599)

In [288]:
# Divide data into features X and target (Classes) Y
X = df.iloc[:,0:num_col-1]
Y = df.iloc[:, num_col-1]
print(X.shape)
print(Y.shape)

(287, 1598)
(287,)


In [289]:
X.describe()

,AP_1,AP_2,AP_3,AP_4,AP_5,AP_6,AP_7,AP_8,AP_9,AP_10,...,AP_1589,AP_1590,AP_1591,AP_1592,AP_1593,AP_1594,AP_1595,AP_1596,AP_1597,AP_1598
count,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,...,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000
mean,-105.909408,-109.905923,-105.184669,-109.846690,-102.613240,-109.132404,-109.850174,-102.797909,-109.909408,-109.961672,...,-106.062718,-105.247387,-109.947735,-109.954704,-109.825784,-102.386760,-109.954704,-107.933798,-106.703833,-109.759582
std,12.432425,1.593760,13.221779,1.534287,17.050529,5.690431,1.498901,17.226005,1.534731,0.649309,...,11.804478,14.160844,0.885422,0.767366,1.825700,17.492063,0.767366,8.111492,10.860499,2.097627
min,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,...,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000
25%,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,...,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000
50%,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,...,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000
75%,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,...,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000,-110.000000
max,-40.000000,-83.000000,-41.000000,-92.000000,-41.000000,-59.000000,-92.000000,-29.000000,-84.000000,-99.000000,...,-47.000000,-35.000000,-95.000000,-97.000000,-85.000000,-41.000000,-97.000000,-68.000000,-55.000000,-86.000000


In [290]:
# Check for class imbalance
print(df.groupby(Y).size())

target
REGION1     19
REGION10     4
REGION11     8
REGION12    17
REGION13     3
REGION14    15
REGION15    24
REGION16     6
REGION17    12
REGION18    21
REGION2     20
REGION3     15
REGION4     18
REGION5     26
REGION6     17
REGION7     11
REGION8     28
REGION9     23
dtype: int64


In [291]:
# Normalize features within range 0 (minimum) and 1 (maximum)
scaler = MinMaxScaler(feature_range=(0, 1))
X_scale = scaler.fit_transform(X)
X_scale = pd.DataFrame(X_scale)

In [292]:
# Encode target values as integers(0~17) from string(REGION1, ... REGION18)
# Y_en = pd.get_dummies(Y)
encoder = LabelEncoder()
encoder.fit(Y)
Y_en = encoder.transform(Y)
Y_en = to_categorical(Y_en)

In [324]:
# pickle.dump(scaler, open('scaler.pkl', 'wb'))
# pickle.dump(encoder, open('label_encoder.pkl', 'wb'))

In [325]:
sc_temp = pickle.load(open('scaler.pkl', 'rb'))
en_temp = pickle.load(open('label_encoder.pkl', 'rb'))

In [326]:
# test scaler saved as pickle file

# sc_temp.transform(np.expand_dims(X.values[0], axis=0))

array([[0.        , 0.        , 0.        , ..., 0.66666667, 0.        ,
        0.        ]])

In [329]:
# test encoder saved as pickle file
# en_temp.transform(['REGION16', 'REGION9'])

array([ 7, 17], dtype=int64)

In [296]:
# For Keras, convert dataframe to array values (Inbuilt requirement of Keras)
X_scale = X_scale.values

In [297]:
X_scale

array([[0.        , 0.        , 0.        , ..., 0.66666667, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.76190476, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.92857143, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.66666667, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.17391304, ..., 0.        , 0.        ,
        0.        ]])

In [298]:
Y_en

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [299]:
# Define baseline model. Then use it in Keras Classifier to implement cross validation
def baseline_model():
    # Create model here
    model = Sequential()
    model.add(Dense(25, input_dim = num_col-1, activation = 'relu')) # Rectified Linear Unit Activation Function
    model.add(Dense(25, activation = 'relu'))
    model.add(Dense(18, activation = 'softmax')) # Softmax for multi-class classification(18 target classes)
    # Compile model here
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [300]:
# Create Keras Classifier and use predefined baseline model
estimator = KerasClassifier(build_fn = baseline_model, epochs = 100, batch_size = 10, verbose = 0)

In [301]:
# KFold Stratified Cross Validation
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

In [302]:
results = cross_val_score(estimator, X_scale, Y, cv = kfold)
# Result
# print("Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

D:\Anaconda3\envs\machinelearning\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [303]:
results # cross-validation result

array([0.93103451, 0.84482759, 0.92982459, 0.92982459, 0.94736844])

In [304]:
estimator.fit(X_scale, Y_en)

In [305]:
estimator.score(X_scale, Y_en)

0.9790940880775452

In [316]:
# scaler.transform(np.expand_dims(X.values[0], axis=0)).squeeze()

array([0.        , 0.        , 0.        , ..., 0.66666667, 0.        ,
       0.        ])

In [358]:
# Prediction test with 1st row of training set
test_data = scaler.transform(np.expand_dims(X.values[7], axis=0)) 

In [362]:
test_data

array([[0.85714286, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [363]:
estimator.predict(test_data)

array([11])

In [364]:
# get prediction result
# estimator.predict(test_data)
result = encoder.inverse_transform(estimator.predict(test_data))[0]

In [365]:
result

'REGION3'

In [332]:
# estimator.save('my_model.hdf5')

AttributeError: 'KerasClassifier' object has no attribute 'save'

In [307]:
X

,AP_1,AP_2,AP_3,AP_4,AP_5,AP_6,AP_7,AP_8,AP_9,AP_10,...,AP_1589,AP_1590,AP_1591,AP_1592,AP_1593,AP_1594,AP_1595,AP_1596,AP_1597,AP_1598
0,-110,-110,-110,-110,-97,-110,-110,-74,-110,-110,...,-110,-110,-110,-110,-110,-110,-110,-82,-110,-110
1,-110,-110,-110,-110,-110,-110,-110,-66,-110,-110,...,-110,-91,-110,-110,-110,-110,-110,-78,-110,-110
2,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-82,-110,-110,-110,-110,-110,-71,-110,-110
3,-110,-110,-110,-110,-110,-110,-110,-61,-110,-110,...,-110,-110,-110,-110,-110,-110,-110,-71,-110,-110
4,-110,-110,-110,-110,-97,-110,-110,-74,-110,-110,...,-110,-110,-110,-110,-110,-110,-110,-82,-110,-110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,-110,-110,-57,-110,-110,-110,-110,-110,-110,-110,...,-95,-110,-110,-110,-110,-110,-110,-110,-110,-110
283,-110,-110,-98,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,-110,-110,-67,-110,-110,-110,-110
284,-110,-110,-64,-110,-110,-110,-110,-110,-110,-110,...,-87,-110,-110,-110,-110,-110,-110,-110,-110,-110
285,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-86,-110,-110,-110,-110,-71,-110,-110,-110,-110


In [327]:
Y

0      REGION16
1      REGION16
2      REGION16
3      REGION16
4      REGION16
         ...   
282     REGION9
283     REGION9
284     REGION9
285     REGION9
286     REGION9
Name: target, Length: 287, dtype: object

In [339]:
# now actually make model
model = Sequential()
model.add(Dense(25, input_dim = num_col-1, activation = 'relu')) # Rectified Linear Unit Activation Function
model.add(Dense(25, activation = 'relu'))
model.add(Dense(18, activation = 'softmax')) # Softmax for multi-class classification(18 target classes)
# Compile model here
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [341]:
model.fit(X_scale, Y_en, epochs = 100, batch_size = 10)

Epoch 1/100
29/29 [==============================] - 0s 552us/step - loss: 2.2633 - accuracy: 0.3136
Epoch 2/100
29/29 [==============================] - 0s 552us/step - loss: 1.5812 - accuracy: 0.6167
Epoch 3/100
29/29 [==============================] - 0s 586us/step - loss: 1.0519 - accuracy: 0.7909
Epoch 4/100
29/29 [==============================] - 0s 552us/step - loss: 0.6732 - accuracy: 0.9024
Epoch 5/100
29/29 [==============================] - 0s 586us/step - loss: 0.4307 - accuracy: 0.9512
Epoch 6/100
29/29 [==============================] - 0s 552us/step - loss: 0.2937 - accuracy: 0.9756
Epoch 7/100
29/29 [==============================] - 0s 586us/step - loss: 0.2166 - accuracy: 0.9756
Epoch 8/100
29/29 [==============================] - 0s 552us/step - loss: 0.1748 - accuracy: 0.9721
Epoch 9/100
29/29 [==============================] - 0s 552us/step - loss: 0.1411 - accuracy: 0.9756
Epoch 10/100
29/29 [==============================] - 0s 586us/step - loss: 0.1159 - accura

In [359]:
actual_result = np.argmax(model.predict(test_data).squeeze()) # predict result

In [360]:
actual_result

11

In [361]:
# result
encoder.inverse_transform([actual_result])[0]

'REGION3'

In [366]:
# model.save('keras_model1')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: keras_model1\assets
